In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_json('/content/drive/MyDrive/dataweave/product_title.json',lines=True)

In [2]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import pprint
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(df1.text.iloc[-1])
print([(X.text, X.label_) for X in doc.ents])

[('Kenneth Cole', 'PERSON'), ("Men's", 'ORG'), ('12', 'CARDINAL')]


In [ ]:
doc = nlp(df1.text.iloc[0])
print([(X.text, X.label_) for X in doc.ents])

[("Propet Logan Men's Sneakers", 'ORG'), ('8', 'CARDINAL')]


In [ ]:
!pip install flair

In [4]:
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger
import logging
import json
#Import flair modules
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, BytePairEmbeddings, TransformerWordEmbeddings

from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [5]:
trained_model_path = '/content/drive/MyDrive/dataweave/best-model.pt'
flair_model =  SequenceTagger.load(trained_model_path)

2022-05-24 13:13:33,110 loading file /content/drive/MyDrive/dataweave/best-model.pt
2022-05-24 13:13:46,571 SequenceTagger predicts: Dictionary with 6 tags: O, B-BRAND, I-BRAND, -, <START>, <STOP>


In [ ]:
sen = Sentence(df1.text.iloc[-1])
flair_model.predict(sen)
sen.to_tagged_string()

'Sentence: "Kenneth Cole Men \'s Stand Tennis-Style Sneakers Black 12" → ["Kenneth Cole"/BRAND]'

In [ ]:
sen = Sentence(df1.text.iloc[0])
flair_model.predict(sen)
sen.to_tagged_string()

'Sentence: "Propet Logan Men \'s Sneakers Black 8 MED" → ["Propet"/BRAND]'

In [113]:
from bs4 import BeautifulSoup
import requests

In [114]:
wikiurl="https://en.wikipedia.org/wiki/List_of_colors:_A%E2%80%93F"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [115]:
soup = BeautifulSoup(response.text, 'html.parser')
color1=soup.find('table',{'class':"wikitable"})

In [116]:
df=pd.read_html(str(color1))
df1=pd.DataFrame(df[0])

In [117]:
wikiurl= "https://en.wikipedia.org/wiki/List_of_colors:_G%E2%80%93M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
soup = BeautifulSoup(response.text, 'html.parser')
color2=soup.find('table',{'class':"wikitable"})
df=pd.read_html(str(color2))
df2=pd.DataFrame(df[0])

wikiurl= "https://en.wikipedia.org/wiki/List_of_colors:_N%E2%80%93Z"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
soup = BeautifulSoup(response.text, 'html.parser')
color3=soup.find('table',{'class':"wikitable"})
df=pd.read_html(str(color3))
df3=pd.DataFrame(df[0])

df_col = pd.concat([df1,df2,df3])

In [118]:
colors = list(df_col.Name)
colors = [col.lower() for col in colors]+['gray']

In [119]:
len(colors)

986

In [120]:
colors

['absolute zero',
 'acid green',
 'aero',
 'aero blue',
 'african violet',
 'air superiority blue',
 'alabaster',
 'alizarin',
 'alice blue',
 'alloy orange',
 'almond',
 'amaranth',
 'amaranth (m&p)',
 'amaranth pink',
 'amaranth purple',
 'amaranth red',
 'amazon',
 'amber',
 'amber (sae/ece)',
 'amethyst',
 'android green',
 'antique brass',
 'antique bronze',
 'antique fuchsia',
 'antique ruby',
 'antique white',
 'ao (english)',
 'apple green',
 'apricot',
 'aqua',
 'aquamarine',
 'arctic lime',
 'army green',
 'artichoke',
 'arylide yellow',
 'ash gray',
 'asparagus',
 'astronaut',
 'atomic tangerine',
 'auburn',
 'aureolin',
 'avocado',
 'azure',
 'azure (x11/web color)',
 'baby blue',
 'baby blue eyes',
 'baby pink',
 'baby powder',
 'baker-miller pink',
 'banana mania',
 'barbie pink',
 'barn red',
 'battleship grey',
 'beau blue',
 'beaver',
 'beige',
 "b'dazzled blue",
 'big dip o’ruby',
 'bisque',
 'bistre',
 'bistre brown',
 'bitter lemon',
 'bitter lime',
 'bittersweet',


In [ ]:
def extract_color(x,colors=colors):
    col_lst = []
    for color in colors:
        if color[::-1] in x.lower()[::-1]:
            col_lst.append(color) 
    return col_lst

In [ ]:
df1['color'] = df1.text.apply(extract_color)

In [ ]:
df1.to_csv('/content/drive/MyDrive/dataweave/df1.csv',index=False)

In [6]:
df1 = pd.read_csv('/content/drive/MyDrive/dataweave/df1.csv')

In [7]:
df1['color'] = df1['color'].apply(eval)

In [8]:
df1['siz_col'] = df1['color'].apply(len)

In [9]:
df1_colorNA = df1[df1['siz_col']==0]

In [10]:
import re
import time

In [11]:
def extract_brand(x,model=flair_model):
    sen = Sentence(str(x))
    model.predict(sen)
    if re.findall(r'\[.*?]', sen.to_tagged_string()):
        return re.findall(r'\[.*?]', sen.to_tagged_string())[-1].split('/BRAND')[0][2:-1]
    else:
        return None

In [ ]:
df_sample = df1.sample(frac = 1).reset_index(drop=True)[:200000]

In [ ]:
start = time.time()
df_sample['brand'] = df_sample.text.apply(extract_brand)
end = time.time()
print(end - start)
df_sample.to_csv('/content/drive/MyDrive/dataweave/df_2Lac.csv',index=False)

2934.2676379680634


In [ ]:
df_2L = pd.read_csv('/content/drive/MyDrive/dataweave/df_2Lac.csv')

In [ ]:
df_all1 = df1.sample(frac = 1).reset_index(drop=True)[200000:500000]
df_all2 = df1.sample(frac = 1).reset_index(drop=True)[500000:800000]
df_all3 = df1.sample(frac = 1).reset_index(drop=True)[800000:1100000]

In [ ]:
start = time.time()
df_all1['brand'] = df_all1.text.apply(extract_brand)
end = time.time()
print(end - start)
df_all1.to_csv('/content/drive/MyDrive/dataweave/df_2-5Lac.csv',index=False)

4343.062129497528


In [ ]:
start = time.time()
df_all2['brand'] = df_all2.text.apply(extract_brand)
end = time.time()
print(end - start)
df_all2.to_csv('/content/drive/MyDrive/dataweave/df_5-8Lac.csv',index=False)

4365.771831512451


In [ ]:
start = time.time()
df_all3['brand'] = df_all3.text.apply(extract_brand)
end = time.time()
print(end - start)
df_all3.to_csv('/content/drive/MyDrive/dataweave/df_8-11Lac.csv',index=False)

4316.483398199081


In [12]:
df_all4 = df1.sample(frac = 1).reset_index(drop=True)[1100000:1500000]
start = time.time()
df_all4['brand'] = df_all4.text.apply(extract_brand)
end = time.time()
print(end - start)
df_all4.to_csv('/content/drive/MyDrive/dataweave/df_11-15Lac.csv',index=False)

5746.5415852069855


In [13]:
df_all5 = df1.sample(frac = 1).reset_index(drop=True)[1500000:]
start = time.time()
df_all5['brand'] = df_all5.text.apply(extract_brand)
end = time.time()
print(end - start)
df_all5.to_csv('/content/drive/MyDrive/dataweave/df_15-20Lac.csv',index=False)

6356.518987894058


In [14]:
df_2L = pd.read_csv('/content/drive/MyDrive/dataweave/df_2Lac.csv')
df_all1 = pd.read_csv('/content/drive/MyDrive/dataweave/df_2-5Lac.csv')
df_all2 = pd.read_csv('/content/drive/MyDrive/dataweave/df_5-8Lac.csv')
df_all3 = pd.read_csv('/content/drive/MyDrive/dataweave/df_8-11Lac.csv')
df_all4 = pd.read_csv('/content/drive/MyDrive/dataweave/df_11-15Lac.csv')
df_all5 = pd.read_csv('/content/drive/MyDrive/dataweave/df_15-20Lac.csv')

In [15]:
df_all = pd.concat([df_2L,df_all1,df_all2,df_all3,df_all4,df_all5])

In [17]:
def lower(x):
    if type(x)==str:
        return x.lower()
    else:
        return x
df_all['brand'] = df_all.brand.apply(lower)

In [23]:
df_all.to_csv('df_all.csv',index=False)

In [22]:
df_all[df_all['brand'].isnull()]

,text,id,color,siz_col,brand
4,Jen 7 Cropped Wide Leg Jeans in Quincy - Quinc...,fadea43e43b873c1f93a06be94d7634b156a378a,[],0,NaN
18,ROCKPORT Women^s Total Motion Adelyn Ballet Fl...,c75b0147dc88d1830886375c4b9912342ecdd876,['black'],1,NaN
30,Men's Nick Graham Traveler Collection Performa...,a491536a09961128706cc66450ef9f29ce7add90,['blue'],1,NaN
42,Under Armour Surge 2 Alt Toddler Kids' Sneaker...,c2df81b5af1250f2f31b91a789ec9eac18ec49ae,['midnight'],1,NaN
45,Bloom 2 Slip-ons Black Microsuede 8.5M,72d45a0870cb4fae81d09d1716bdbfe20b6afd1c,['black'],1,NaN
...,...,...,...,...,...
443807,"4"" Wedge Sole Boot DUST BROWN 9",80d37026e665e13f7bafe70e90ffb66a67879afd,['brown'],1,NaN
443810,J. Reneé Ramillo Slingback Peep Toe Pump VINYL...,60a3aa305405113bdb5bd90fb064f441f1878f2b,[],0,NaN
443812,J. Reneé Leone Slingback Crystal Embellished S...,0eaed4c3671d488811b42d257696cf3080efc473,"['black', 'crystal', 'sand']",3,NaN
443814,Men's Cozy Center Slipper Black 9M,5707b79ed5fc0f7c5d2c1b6f255003597d5f470d,['black'],1,NaN


In [38]:
df_all['text_N'] = df_all['text'].apply(lambda x: ' '.join(str(x).upper().split(' ')[-5:]))

In [39]:
distribution = df_all.text_N.str.split(expand=True).stack().value_counts()

In [97]:
size_lst = ['M','W','S','L','XL','N','XS','XXL','D','XXS','XXW','XX','S/M','L/XL','M/L','XS/S','XXXL','XL/REG','M/REG','L/REG','S/REG','REG','XS/REG']

In [98]:
size_wrd = ['MEDIUM','WIDE','REGULAR','NARROW','SMALL','EXTRA','LARGE']

In [99]:
def get_size(x,size_lst = size_lst,size_wrd = size_wrd):
    fin_lst = []
    str_lst = str(x).split(' ')
    for st in str_lst:
        if re.findall('\d*\.?\d+',st):
            fin_lst.append(st)
        if st in size_lst:
            fin_lst.append(st)
    for wrd in size_wrd:
        if wrd in str(x).upper():
            fin_lst.append(wrd)
    return fin_lst
df_all['size'] = df_all['text'].apply(get_size)

In [101]:
df_all['siz_size'] = df_all['size'].apply(len)

In [102]:
df_all_textNA = df_all[df_all.siz_size==0]

In [109]:
data_fin =  df_all[['text','id','color','brand','size']]
data_fin.to_csv('/content/drive/MyDrive/dataweave/data_final.csv',index=False)

In [110]:
df_all_ = pd.read_csv('/content/drive/MyDrive/dataweave/data_final.csv')

In [112]:
df_all_

,text,id,color,brand,size
0,adidas Originals Trefoil Hoodie - Men's - XL -...,d9c5968b4ab2e15e10a836901f33456722a24cec,"['white', 'gray']",adidas originals,['XL']
1,Helga Sandal Dark Brown Leather 7 Medium,e9406911dbccd97b384ac72c768c6cc1c0ad7a1b,"['brown', 'dark brown', 'sand']",helga,"['7', 'MEDIUM']"
2,Women’s Osito Jacket | Roxbury Pink | XL,8730d00193813cd74ee91dcb8a5364d7cb1e0bbc,['pink'],roxbury,['XL']
3,"Women's Harley-Davidson Bremerton 14"" Buckles ...",e279ef47a34a8da232147e9dfcc7962cf18b47f0,['black'],bremerton,"['14""', '8.5Black', 'M', 'REGULAR']"
4,Jen 7 Cropped Wide Leg Jeans in Quincy - Quinc...,fadea43e43b873c1f93a06be94d7634b156a378a,[],NaN,"['7', '10', 'WIDE']"
...,...,...,...,...,...
1943813,Marianne Slip-On Sneaker Black Perforated Leat...,4259c77b34b84bdff5e14e9f69dfa309338aa3b4,['black'],marianne slip-on,"['9', 'WIDE', 'EXTRA']"
1943814,Men's Cozy Center Slipper Black 9M,5707b79ed5fc0f7c5d2c1b6f255003597d5f470d,['black'],NaN,['9M']
1943815,PUMA Suede Classic XXI Sneaker PUMA BLACK/ PUM...,d97c45ff0acb1d30305ea734105fe824d6e3a934,"['black', 'white']",NaN,['6']
1943816,adidas Cloudfoam Puremotion Women's Running Sh...,a08c02f33188798a7831269a1709fbaf4a06c9bd,['purple'],adidas,['8.5']
